In [2]:
import re
import pandas as pd
import numpy as np


In [3]:
with open('GOOG.txt', 'r') as f:
    data = f.read()


# Split into documents
documents = data.replace('<DOCUMENT>', '').split('</DOCUMENT>')

len(documents)


115

In [4]:
reg_FILE = re.compile('FILE:(.*)')
reg_TIME = re.compile('TIME:(.*)')
reg_EVENTS = re.compile('EVENTS:(.*)')
reg_ITEMS = re.compile('ITEM:(.*)')

def factory_parser(compiled_reg):
    return lambda d: map(lambda x: x.replace(',', '').strip(), re.findall(compiled_reg, d))

get_file = factory_parser(reg_FILE)
get_time = factory_parser(reg_TIME)
get_events = factory_parser(reg_EVENTS)
get_items = factory_parser(reg_ITEMS)



META_LINES = ['FILE', 'TIME', 'EVENTS', 'ITEM']

def get_text(doc):
    lines = [ ln for ln in doc.split('\n') if ln != ' ' ]
    g_lines = [ ln for ln in lines if not any(map(lambda x: ln.startswith(x), META_LINES)) ]
    return '\n'.join( g_lines )




# Parse documents
parsed_data = {
    'file': map(get_file, documents),
    'time': map(get_time, documents),
    'events': map(get_events, documents),
    'items': map(get_items, documents),
    'text': map(get_text, documents)
 }



struct_data = pd.DataFrame.from_dict(parsed_data)


struct_data.head()



,events,file,items,text,time
0,[Other events\tFinancial statements and exhibits],[GOOG/GOOG-8K-20040709060939.txt.gz],"[Other events, Financial statements and exhibits]","\nTEXT:\nOn July 6, 2004, Google Inc. (""Google...",[20040709060939]
1,[Other Events\tFinancial Statements and Exhibits],[GOOG/GOOG-8K-20041008171511.txt.gz],"[Other Events, Financial Statements and Exhibits]",\n\nTEXT:\nCheck the appropriate box below if ...,[20041008171511]
2,[Results of Operations and Financial Condition...,[GOOG/GOOG-8K-20041021162846.txt.gz],[Results of Operations and Financial Condition...,\n\nTEXT:\nCheck the appropriate box below if ...,[20041021162846]
3,[Other Events],[GOOG/GOOG-8K-20041119172648.txt.gz],[Other Events],\n\nTEXT:\nCheck the appropriate box below if ...,[20041119172648]
4,[Other Events\tFinancial Statements and Exhibits],[GOOG/GOOG-8K-20050113122048.txt.gz],"[Other Events, Financial Statements and Exhibits]",\n\nTEXT:\nCheck the appropriate box below if ...,[20050113122048]


In [30]:
re.sub("[^0-9]", "", "sdkjh987978asd098as0980a98sd")

'987978098098098'

In [5]:


time_list = struct_data.time
# print (only_numerics(time_list[0]))
re.sub("[^0-9]", "", str(time_list[3]))
text_list = struct_data.text
print(text_list[0])


TEXT:
On July 6, 2004, Google Inc. ("Google") filed its Second Amended and Restated Certificate of Incorporation with the Secretary of State of the State of Delaware. This Second Amended and Restated Certificate of Incorporation was duly approved and adopted by Google's board of directors and by its stockholders on June 25, 2004. Google's Second Amended and Restated Certificate of Incorporation implements certain changes with respect to Google's dual class common stock structure and provides for certain corporate governance requirements that will be applicable to Google once it become a public company.
For more information regarding these amendments please see the full text of the Second Amended and Restated Certificate of Incorporation, which is attached as Exhibit 3.01 to this report.
(c)  Exhibits
Pursuant to the requirements of the Securities Exchange Act of 1934, the Registrant has duly caused this Report to be signed on its behalf by the undersigned hereunto duly authorized.

EX

In [7]:


def closet_date_info(how_many, date, data_frame):
    date_num = int(date) * 1000000
    closet = 0
    time_num = int(re.sub("[^0-9]", "", str(data_frame.time[closet])))
    final_text = ""
    if (date_num <= time_num):
         print ("Invalid date due to out of data records range.")
    else: 
        while (time_num < date_num):
            closet = closet + 1
            time_num = int(re.sub("[^0-9]", "", str(data_frame.time[closet])))
        for i in range(0, how_many):
            if ((closet  - 1 - i) >= 0):
                #print(data_frame.time[(closet  - 1 - i)])    
                #print(data_frame.text[(closet  - 1 - i)]) 
                final_text =  final_text + data_frame.text[(closet  - 1 - i)]
            else:
                print ("ATTENTION!!!!!!!!!!!!!!!Only above financial report available.")
                break;
    return final_text
                


### Please input date as 20080101 format, first four are years, second two are months, last two are date
# how_closet (between 1 and 5), enter the number of closet news
# return time (to secs) and info (text format) 
#closet_date_info(how_closet, date, data_frame)
closet_date_info(3, 20051001, struct_data)




'\n\nTEXT:\nCheck the appropriate box below if the Form 8-K filing is intended to simultaneously satisfy the filing obligation of the registrant under any of the following provisions (see General Instruction A.2. below):\nItem 2.02. Results of Operations and Financial Condition\nOn July 21, 2005, Google is issuing a press release and holding a conference call regarding its financial results for the second quarter of fiscal 2005 ended June 30, 2005. A copy of the press release is furnished as Exhibit 99.1 to this Form 8-K.\nThis information shall not be deemed "filed" for purposes of Section 18 of the Securities Exchange Act of 1934, as amended (the "Exchange Act"), or incorporated by reference in any filing under the Securities Act of 1933, as amended, or the Exchange Act, except as shall be expressly set forth by specific reference in such a filing.\nGoogle is making reference to non-GAAP financial information in both the press release and the conference call. A reconciliation of thes

In [12]:

import io
import nltk
import itertools
from operator import itemgetter
import networkx as nx
import os

#apply syntactic filters based on POS tags
def filter_for_tags(tagged, tags=['NN', 'JJ', 'NNP']):
    return [item for item in tagged if item[1] in tags]

def normalize(tagged):
    return [(item[0].replace('.', ''), item[1]) for item in tagged]

def unique_everseen(iterable, key=None):
    "List unique elements, preserving order. Remember all elements ever seen."
    # unique_everseen('AAAABBBCCDAABBB') --> A B C D
    # unique_everseen('ABBCcAD', str.lower) --> A B C D
    seen = set()
    seen_add = seen.add
    if key is None:
        for element in itertools.ifilterfalse(seen.__contains__, iterable):
            seen_add(element)
            yield element
    else:
        for element in iterable:
            k = key(element)
            if k not in seen:
                seen_add(k)
                yield element

def lDistance(firstString, secondString):
    "Function to find the Levenshtein distance between two words/sentences - gotten from http://rosettacode.org/wiki/Levenshtein_distance#Python"
    if len(firstString) > len(secondString):
        firstString, secondString = secondString, firstString
    distances = range(len(firstString) + 1)
    for index2, char2 in enumerate(secondString):
        newDistances = [index2 + 1]
        for index1, char1 in enumerate(firstString):
            if char1 == char2:
                newDistances.append(distances[index1])
            else:
                newDistances.append(1 + min((distances[index1], distances[index1+1], newDistances[-1])))
        distances = newDistances
    return distances[-1]

def buildGraph(nodes):
    "nodes - list of hashables that represents the nodes of the graph"
    gr = nx.Graph() #initialize an undirected graph
    gr.add_nodes_from(nodes)
    nodePairs = list(itertools.combinations(nodes, 2))

    #add edges to the graph (weighted by Levenshtein distance)
    for pair in nodePairs:
        firstString = pair[0]
        secondString = pair[1]
        levDistance = lDistance(firstString, secondString)
        gr.add_edge(firstString, secondString, weight=levDistance)

    return gr

def extractKeyphrases(text):
    # tokenize the text using nltk
    wordTokens = nltk.word_tokenize(text)

    # assign POS tags to the words in the text
    tagged = nltk.pos_tag(wordTokens)
    textlist = [x[0] for x in tagged]
    
    tagged = filter_for_tags(tagged)
    tagged = normalize(tagged)

    unique_word_set = unique_everseen([x[0] for x in tagged])
    word_set_list = list(unique_word_set)

    # this will be used to determine adjacent words in order to construct keyphrases with two words
    graph = buildGraph(word_set_list)
    # pageRank - initial value of 1.0, error tolerance of 0,0001, 
    calculated_page_rank = nx.pagerank(graph, weight='weight')
    print (calculated_page_rank.items()[:4])
    # most important words in ascending order of importance
    keyphrases = sorted(calculated_page_rank, key=calculated_page_rank.get, reverse=True)
    #print (len(keyphrases))
    # the number of keyphrases returned will be relative to the size of the text (a third of the number of vertices)
    aThird = len(word_set_list) / 3
    keyphrases = keyphrases[0:aThird+1]
    # take keyphrases with multiple words into consideration as done in the paper - if two words are adjacent 
    # in the text and are selected as keywords, join them together
    modifiedKeyphrases_values = {}
    dealtWith = set([]) #keeps track of individual keywords that have been joined to form a keyphrase
    i = 0
    j = 1
    while j < len(textlist):
        firstWord = textlist[i]
        secondWord = textlist[j]
        if firstWord in keyphrases and secondWord in keyphrases:
            keyphrase = firstWord + ' ' + secondWord
            modifiedKeyphrases.add(keyphrase)
            dealtWith.add(firstWord)
            dealtWith.add(secondWord)
        else:
            if firstWord in keyphrases and firstWord not in dealtWith: 
                modifiedKeyphrases.add(firstWord)
            #if this is the last word in the text, and it is a keyword,
            #it definitely has no chance of being a keyphrase at this point    
            if j == len(textlist)-1 and secondWord in keyphrases and secondWord not in dealtWith:
                modifiedKeyphrases.add(secondWord)      
        i = i + 1
        j = j + 1
        
    return  calculated_page_rank, modifiedKeyphrases

def extractSentences(text):
    sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
    sentenceTokens = sent_detector.tokenize(text.strip())
    graph = buildGraph(sentenceTokens)

    calculated_page_rank = nx.pagerank(graph, weight='weight')

    #most important sentences in ascending order of importance
    sentences = sorted(calculated_page_rank, key=calculated_page_rank.get, reverse=True)

    #return a 100 word summary
    summary = ' '.join(sentences)
    summaryWords = summary.split()
    summaryWords = summaryWords[0:101]
    summary = ' '.join(summaryWords)

    return summary

def writeFiles(summary, keyphrases, fileName):
    "outputs the keyphrases and summaries to appropriate files"
    print "Generating output to " + 'TextRank-master/keywords/' + fileName
    keyphraseFile = io.open('TextRank-master/keywords/' + fileName, 'w')
    for keyphrase in keyphrases:
        keyphraseFile.write(keyphrase + '\n')
    keyphraseFile.close()

    print "Generating output to " + 'TextRank-master/summaries/' + fileName
    summaryFile = io.open('TextRank-master/summaries/' + fileName, 'w')
    summaryFile.write(summary)
    summaryFile.close()

    print "-"

def mainRunner():
    #nltk.download()
    #retrieve each of the articles
    articles = os.listdir("TextRank-master/articles")
    for article in articles:
        print 'Reading articles/' + article
        articleFile = io.open('TextRank-master/articles/' + article, 'r')
        text = articleFile.read()
        keyphrases = extractKeyphrases(text)
        summary = extractSentences(text)
        writeFiles(summary, keyphrases, article)

In [13]:
'''
NO rank value version

'''

import io
import nltk
import itertools
from operator import itemgetter
import networkx as nx
import os

positive_words = {'grow', 'growth', 'increase', 'increases', 'new', 'strong', 'forward', 'well', 
                  'increament', 'boost', 'consolidation', 'expand', 'expansion', 'fuel', 'gain', 'meager',
                  'good', 'better', 'beneficiary', 'google'}
negative_words = {'loss', 'decrease', 'decreases', 'lower', 'reduce', 'down', 'reduction', 
                  'decline', 'reduction', 'decreament', 'bailout', 'bubble', 'carryover', 'cautious', 
                  'charge', 'debt', 'drop', 'flat', 'inflation', 'jobless', 'recession', 'bad', 'worse', 
                 'shrink', 'slow', 'slump', 'tight', 'umemployment', 'wane'}
polarity_weight = 0

#apply syntactic filters based on POS tags
def filter_for_tags(tagged, tags=['NN', 'JJ', 'NNP']):
    return [item for item in tagged if item[1] in tags]

def normalize(tagged):
    return [(item[0].replace('.', ''), item[1]) for item in tagged]

def unique_everseen(iterable, key=None):
    "List unique elements, preserving order. Remember all elements ever seen."
    # unique_everseen('AAAABBBCCDAABBB') --> A B C D
    # unique_everseen('ABBCcAD', str.lower) --> A B C D
    seen = set()
    seen_add = seen.add
    if key is None:
        for element in itertools.ifilterfalse(seen.__contains__, iterable):
            seen_add(element)
            yield element
    else:
        for element in iterable:
            k = key(element)
            if k not in seen:
                seen_add(k)
                yield element

def lDistance(firstString, secondString):
    "Function to find the Levenshtein distance between two words/sentences - gotten from http://rosettacode.org/wiki/Levenshtein_distance#Python"
    if len(firstString) > len(secondString):
        firstString, secondString = secondString, firstString
    distances = range(len(firstString) + 1)
    for index2, char2 in enumerate(secondString):
        newDistances = [index2 + 1]
        for index1, char1 in enumerate(firstString):
            if char1 == char2:
                newDistances.append(distances[index1])
            else:
                newDistances.append(1 + min((distances[index1], distances[index1+1], newDistances[-1])))
        distances = newDistances
    return distances[-1]

def buildGraph_weight(nodes):
    "nodes - list of hashables that represents the nodes of the graph"
    gr = nx.Graph() #initialize an undirected graph
    gr.add_nodes_from(nodes)
    nodePairs = list(itertools.combinations(nodes, 2))

    #add edges to the graph (weighted by Levenshtein distance)
    for pair in nodePairs:
        firstString = pair[0]
        secondString = pair[1]
        #print ("First String", firstString)
        #print ("Second String", secondString)
        if (firstString.lower() in positive_words or secondString.lower() in positive_words or 
           firstString.lower() in negative_words or secondString.lower() in negative_words):
                levDistance = polarity_weight * lDistance(firstString, secondString)
        else:
            levDistance = lDistance(firstString, secondString)
        gr.add_edge(firstString, secondString, weight=levDistance)

    return gr

def buildGraph_no_weight(nodes):
    "nodes - list of hashables that represents the nodes of the graph"
    gr = nx.Graph() #initialize an undirected graph
    gr.add_nodes_from(nodes)
    nodePairs = list(itertools.combinations(nodes, 2))

    #add edges to the graph (weighted by Levenshtein distance)
    for pair in nodePairs:
        firstString = pair[0]
        secondString = pair[1]
        levDistance = lDistance(firstString, secondString)
        gr.add_edge(firstString, secondString, weight=levDistance)

    return gr

def extractKeyphrases(text, weight_indicator):
    # tokenize the text using nltk
    wordTokens = nltk.word_tokenize(text)

    # assign POS tags to the words in the text
    tagged = nltk.pos_tag(wordTokens)
    textlist = [x[0] for x in tagged]
    
    tagged = filter_for_tags(tagged)
    tagged = normalize(tagged)

    unique_word_set = unique_everseen([x[0] for x in tagged])
    word_set_list = list(unique_word_set)

    # this will be used to determine adjacent words in order to construct keyphrases with two words
    if weight_indicator == 1:
        graph = buildGraph_weight(word_set_list)
    else:
        graph = buildGraph_no_weight(word_set_list)
    # pageRank - initial value of 1.0, error tolerance of 0,0001, 
    calculated_page_rank = nx.pagerank(graph, weight='weight')
    # most important words in ascending order of importance
    keyphrases = sorted(calculated_page_rank, key=calculated_page_rank.get, reverse=True)
    # the number of keyphrases returned will be relative to the size of the text (a third of the number of vertices)
    aThird = len(word_set_list) / 3
    keyphrases = keyphrases[0:aThird+1]
    # take keyphrases with multiple words into consideration as done in the paper - if two words are adjacent 
    # in the text and are selected as keywords, join them together
    modifiedKeyphrases = set([])
    dealtWith = set([]) #keeps track of individual keywords that have been joined to form a keyphrase
    i = 0
    j = 1
    while j < len(textlist):
        firstWord = textlist[i]
        secondWord = textlist[j]
        if firstWord in keyphrases and secondWord in keyphrases:
            keyphrase = firstWord + ' ' + secondWord
            modifiedKeyphrases.add(keyphrase)
            dealtWith.add(firstWord)
            dealtWith.add(secondWord)
        else:
            if firstWord in keyphrases and firstWord not in dealtWith: 
                modifiedKeyphrases.add(firstWord)
            #if this is the last word in the text, and it is a keyword,
            #it definitely has no chance of being a keyphrase at this point    
            if j == len(textlist)-1 and secondWord in keyphrases and secondWord not in dealtWith:
                modifiedKeyphrases.add(secondWord)      
        i = i + 1
        j = j + 1
        
    return calculated_page_rank, modifiedKeyphrases

def extractSentences(text):
    sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
    sentenceTokens = sent_detector.tokenize(text.strip())
    graph = buildGraph(sentenceTokens)

    calculated_page_rank = nx.pagerank(graph, weight='weight')

    #most important sentences in ascending order of importance
    sentences = sorted(calculated_page_rank, key=calculated_page_rank.get, reverse=True)

    #return a 100 word summary
    summary = ' '.join(sentences)
    summaryWords = summary.split()
    summaryWords = summaryWords[0:101]
    summary = ' '.join(summaryWords)

    return summary

def writeFiles(summary, keyphrases, fileName):
    "outputs the keyphrases and summaries to appropriate files"
    print "Generating output to " + 'TextRank-master/keywords/' + fileName
    keyphraseFile = io.open('TextRank-master/keywords/' + fileName, 'w')
    for keyphrase in keyphrases:
        keyphraseFile.write(keyphrase + '\n')
    keyphraseFile.close()

    print "Generating output to " + 'TextRank-master/summaries/' + fileName
    summaryFile = io.open('TextRank-master/summaries/' + fileName, 'w')
    summaryFile.write(summary)
    summaryFile.close()

    print "-"

def mainRunner():
    #nltk.download()
    #retrieve each of the articles
    articles = os.listdir("TextRank-master/articles")
    for article in articles:
        print 'Reading articles/' + article
        articleFile = io.open('TextRank-master/articles/' + article, 'r')
        text = articleFile.read()
        keyphrases = extractKeyphrases(text)
        summary = extractSentences(text)
        writeFiles(summary, keyphrases, article)

In [14]:
text = closet_date_info(1, 20060801, struct_data)
#text = "This plane makes aviation easyUS military unveils futuristic helicopterRemembering China's first female jet pilotThe future of airport securityVets reunite with retired planes in 'Boneyard'How airline meals went gourmetJapan's new stealth fighter jetDuck causes a stir by flying on a commercial flightChina successfully docks spacecraftOn board the Charles de GaulleFive new planes you may fly soonInside the Airbus A380 assembly lineWhat we know about the Airbus A320Will we be flying in this in 2030?World's largest plane makes historic landingHow do you refuel a U.S. fighter jet up in the air?Pilot: Solar-powered plane proving the impossibleJapan's new stealth fighter jet takes flightRussia denies jet barrel-rolled over U.S. aircraftArabic-speaking student kicked off flightRussian jet barrel-rolls over U.S. planeFlight attendant shocks passengers by doing this ...Plane crashes into car, kills womanIs this the most annoying thing you can do on a plane? Video said to show moment of impactVintage warplane used to bomb ISISCrazy things people bring on planesFinal flight of an airline icon: The first Boeing 727U.S. military's new vertical takeoff aircraft 5 things you didn't know about the 747World-record pilot takes on educationDaredevils' incredible stunt goes viralSee Boeing Dreamliner built in hyperspeed This futuristic jet engine is very realFuture of flying, with X-ray visionWhat is it like to fly in an F-16? Seriously, who does this on a flight?!American to begin 787 Dreamliner serviceSneak peek into the plane of the futureWhat the future of travel looks likeStory highlightsThe intuitive amphibious ICON A5 looks like a sports carIt does require a sports pilot license to fly it Tampa, Florida (CNN)When it comes to piloting a plane, I have zero experience.In fact, I've never even considered attempting it. Nevertheless, as we swoop over Florida's Tampa Bay on a gorgeous November morning, I enthusiastically take over the controls of the ICON A5, a two-seater amphibious plane that looks like a sports car, maneuvers like a jet ski and is so intuitive, the company says that even a novice can learn to fly it in less than 30 hours.At 23 feet long and weighing in a 1,510 pounds, it's known as a sea light-sport aircraft.Designed to help you focus on the freedom of flying without worrying if the plane will react unpredictably, there's nothing else quite like it on the market.Read MoreNow that I'm in control, my whole body tenses for a good five minutes. Am I really ready to pilot this shiny new machine all by myself? I'm not entirely sure.However, I take comfort in the fact that the A5 was built specifically for people like me, and it was designed to drive like a car. Plus, the fact that Icon's CEO and founder, Kirk Hawkins, is next to me in the cockpit doesn't hurt either. He can take over the controls at any time.So far, so good.We glide smoothly through the air at about 1,000 feet, and I'm comfortable enough to take in the sights. Lovely Fort De Soto Park doesn't disappoint."
text_rank_weight, keyphrases_weight = extractKeyphrases(text, 1)
print (keyphrases_weight)
text_rank_no_weight, keyphrases_no_weight = extractKeyphrases(text, 0)
#summary = extractSentences(text)
print (keyphrases_weight - keyphrases_no_weight)
print (keyphrases_no_weight - keyphrases_weight)
#print (summary)

set(['Reconciliation', '888-203-1112', 'forecasting', 'settlement', 'Non-GAAP', 'forward-looking', 'period-to-period', 'operational', 'Murchinson 650-253-4437', 'Acquisition', 'Stock-Based Compensation', 'quarterly', '650-253-7663', 'presentation', 'confirmation', 'technology infrastructure', 'expectation', 'Analysis', 'anticipated', 'indicative', 'stock-based compensation', 'non-employee', 'Operations', 'impressive', 'headcount-related', 'indication', 'WEBCAST', 'Management', 'financial', 'reference', 'foreseeable', 'consistent', 'Exhibits', 'perspective', 'International', 'registrant', 'substitute', 'CONFERENCE', 'MOUNTAIN', 'financial information', '719-457-0820', 'supplemental information', 'meaningful supplemental', 'available', 'INFORMATION', 'comparable', 'infrastructure', 'historical performance', 'processing', 'Securities', 'limitation', 'marketable', 'remainder', 'information technology', 'Additional information', 'Financial Highlights', 'NON-GAAP FINANCIAL', 'LOOKING STATEME

In [18]:
print(text_rank_weight)

print(text_rank_no_weight)

{'code': 0.002358035926891234, 'TEXT': 0.002617925564019833, 'focus': 0.00249969710157396, 'Schmidt': 0.002716828864173453, 'period-to-period': 0.004678644109316347, 'charge': 0.00042692520525941215, 'behalf': 0.002585100924868182, 'consolidated': 0.003592324056391937, 'quarterly': 0.0029921695518498347, 'include': 0.0026777351163520613, 'presentation': 0.003538377409760016, '@': 0.0025953722011148746, 'confirmation': 0.0035433686880148936, 'MEASURES': 0.003107487877103367, 'Acquisition': 0.003408165933496495, 'impressive': 0.003176836741339199, 'VIEW': 0.0026248622028629425, 'non-GAAP': 0.0029985184730843308, 'consistent': 0.0031068699462366784, 'second': 0.0024916586133452716, 'cost': 0.0023490220321295326, 'perspective': 0.0033222310122580177, 'investment': 0.003142587037704225, 'substitute': 0.0032064758924464517, 'reconciliation': 0.004044452022146329, 'settlement': 0.003185988864807549, 'Form': 0.002453173679143796, 'section': 0.0025270237907712304, 'capital': 0.00263199567635824

In [15]:
def is_positive(word):
    with open('positive_words.txt') as f:
        found = False
        for line in f:
            if word.upper() in line: # Key line: check if `w` is in the line.
                found = True
                return found 
            else:
                continue
        return False
    
def is_negative(word):
    with open('negative_words.txt') as f:
        found = False
        for line in f:
            if word.upper() in line: # Key line: check if `w` is in the line.
                found = True
                return found 
            else:
                continue
        return False

In [16]:
d = text_rank_weight
positive_score = 0
positive_count = 0
negative_score = 0
negative_count = 0
su = 0
for k,v in d.iteritems(): 
    k = k.encode('ascii', 'ignore').decode('ascii')
    su = su + v 
    if k.isalpha() and (is_positive(str(k))):
        #print "%s - %f" % (str(k), v)
        positive_score = positive_score + v
        positive_count = positive_count + 1
        
    elif k.isalpha() and (is_negative(str(k))):
        negative_score = negative_score + v
        negative_count = negative_count + 1
                      
    else:
        continue
print su, positive_score, negative_score, positive_count, negative_count, \
(positive_score/negative_score), (float(positive_count)/float(negative_count)), 

1.0 0.0674671683088 0.146910266721 27 55 0.459240663124 0.490909090909


## Function to return latest text score from k8 Data given date
## text score here is defined as positive_score minus negative_score, score are calculated
## from text rank algorithm. positive and negative words  are determined by dictionary 
## positive_words.txt and negative_words.txt

In [24]:
def k8_score(date, company_data, weight_indicator):
    text = closet_date_info(1, date, company_data)
    if weight_indicator == 0:
        text_rank_no_weight, keyphrases_no_weight = extractKeyphrases(text, 0)
        d = text_rank_no_weight
    else:
        text_rank_weight, keyphrases_weight = extractKeyphrases(text, 1)
        d = text_rank_weight
    
    positive_score = 0
    positive_count = 0
    negative_score = 0
    negative_count = 0   
    su = 0
    for k,v in d.iteritems(): 
        k = k.encode('ascii', 'ignore').decode('ascii')
        su = su + v 
        if k.isalpha() and (is_positive(str(k))):
        #print "%s - %f" % (str(k), v)
            positive_score = positive_score + v
            positive_count = positive_count + 1
        elif k.isalpha() and (is_negative(str(k))):
            negative_score = negative_score + v
            negative_count = negative_count + 1
                      
        else:
            continue
            
    return (positive_score-negative_score)
    
    
    
    
date = 20051001
google_struct_data = struct_data
company_data = google_struct_data
weight_indicator = 0
k8_score(date, company_data, weight_indicator)

-0.06472610734763437